This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [109]:
import sys
print(sys.executable)
print(sys.version)

c:\Users\Ralph\Desktop\ANN\tf-env\Scripts\python.exe
3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]


In [110]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, callbacks, optimizers
import numpy as np


In [111]:
# DATASET DIRECTORY CONFIGURATION (updated to use your pizzavsicecreamdataset)
train_dir = r"C:\Users\Ralph\Documents\GitHub\25-26\CS 3B\RALPH_MARTIN_CHUA\ANN\pizzavsicecreamdataset\train"
test_dir  = r"C:\Users\Ralph\Documents\GitHub\25-26\CS 3B\RALPH_MARTIN_CHUA\ANN\pizzavsicecreamdataset\test"



In [112]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [113]:
# DATA PREPROCESSING & AUGMENTATION (ensure labels match pizza/icecream)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print("Class indices:", train_generator.class_indices)
print(f"Train samples: {train_generator.samples}, Val samples: {val_generator.samples}, Test samples: {test_generator.samples}")

Found 575 images belonging to 2 classes.
Found 143 images belonging to 2 classes.
Found 107 images belonging to 2 classes.
Class indices: {'icecream': 0, 'pizza': 1}
Train samples: 575, Val samples: 143, Test samples: 107


In [114]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION, BATCHNORM, AND DROPOUT

# hyperparams
initial_learning_rate = 0.001
weight_decay = 1e-4

# LR schedule + optimizer
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True,
)
optimizer = optimizers.Adam(learning_rate=lr_schedule)

# Improved model: L2, BatchNorm, Dropout
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding='same',
                  kernel_regularizer=regularizers.l2(weight_decay),
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), activation=None),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), padding='same',
                  kernel_regularizer=regularizers.l2(weight_decay), activation=None),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), padding='same',
                  kernel_regularizer=regularizers.l2(weight_decay), activation=None),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.30),

    layers.Flatten(),
    layers.Dense(256, kernel_regularizer=regularizers.l2(weight_decay), activation=None),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.Dropout(0.5),

    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 batch_normalization_24 (Bat  (None, 128, 128, 32)     128       
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 128, 128, 32)      0         
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 64, 64, 32)        0         
                                                                 
 conv2d_22 (Conv2D)          (None, 64, 64, 64)       

In [115]:
# TRAIN / SAVE (use your lastname in the filename)
MODEL_FILENAME = 'exercise_6_custom_lastname.h5'  # replace custom_lastname with your last name if desired

checkpoint_path = MODEL_FILENAME
callbacks_list = [
    callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
]

history = model.fit(
    train_generator,
    epochs=12,
    validation_data=val_generator,
    callbacks=callbacks_list,
    verbose=1
)

# ensure final save (ModelCheckpoint already saved best model)
model.save(MODEL_FILENAME)
print(f"Saved improved model to {MODEL_FILENAME}")

Epoch 1/12
18/18 [==============================] - ETA: 0s - loss: 0.7285 - accuracy: 0.7930
Epoch 1: val_accuracy improved from -inf to 0.54545, saving model to exercise_6_custom_lastname.h5
18/18 [==============================] - 35s 2s/step - loss: 0.7285 - accuracy: 0.7930 - val_loss: 1.8111 - val_accuracy: 0.5455
Epoch 2/12
18/18 [==============================] - ETA: 0s - loss: 0.4704 - accuracy: 0.8435
Epoch 2: val_accuracy did not improve from 0.54545
18/18 [==============================] - 20s 1s/step - loss: 0.4704 - accuracy: 0.8435 - val_loss: 1.2165 - val_accuracy: 0.5455
Epoch 3/12
18/18 [==============================] - ETA: 0s - loss: 0.3853 - accuracy: 0.8887
Epoch 3: val_accuracy did not improve from 0.54545
18/18 [==============================] - 21s 1s/step - loss: 0.3853 - accuracy: 0.8887 - val_loss: 1.3957 - val_accuracy: 0.5455
Epoch 4/12
18/18 [==============================] - ETA: 0s - loss: 0.3390 - accuracy: 0.8870
Epoch 4: val_accuracy did not improv

In [116]:
# Load best-saved model and evaluate
from tensorflow.keras.models import load_model
MODEL_FILENAME = 'exercise_6_custom_lastname.h5'  # same name used above

model = load_model(MODEL_FILENAME)
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_acc:.4f}")

4/4 [==============================] - 5s 1s/step - loss: 1.7564 - accuracy: 0.5514
Test Accuracy: 0.5514


In [117]:
# SAVE THE MODEL
model.save('muffin_vs_chihuahua_cnn.h5')

In [118]:
# PREDICTION + WRITE RESULTS (updated labels and fixed filename typo)
MODEL_FILENAME = 'exercise_6_custom_lastname.h5'  # change custom_lastname -> your lastname if desired

def predict_image_local(img_path, model_path=MODEL_FILENAME):
    model = tf.keras.models.load_model(model_path)
    from tensorflow.keras.preprocessing import image
    import numpy as np
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = float(model.predict(img_array)[0,0])
    label = "pizza" if pred >= 0.5 else "icecream"   # matches your dataset
    print(f"{img_path} -> Prediction: {label} (confidence: {pred:.4f})")
    return label, pred

# example test images from your dataset (adjust filenames if different)
label1, conf1 = predict_image_local(r"pizzavsicecreamdataset/test/icecream/0072_jpg.rf.aec373e259dc03153312c64dd0bb9b4d.jpg")
label2, conf2 = predict_image_local(r"pizzavsicecreamdataset/test/pizza/00100_jpg.rf.743db856627866dbc7c6d3af90afd0e2.jpg")  # replace with an actual muffin test image

# write results (requires test_acc from evaluation cell)
with open('training_results.txt', 'w') as f:
    f.write(f"Test Accuracy: {test_acc:.4f}\n")
    f.write(f"Run 1 - {label1} (confidence: {conf1:.4f})\n")
    f.write(f"Run 2 - {label2} (confidence: {conf2:.4f})\n")

print("Predictions saved to training_results.txt")

1/1 [==============================] - 1s 562ms/step
pizzavsicecreamdataset/test/icecream/0072_jpg.rf.aec373e259dc03153312c64dd0bb9b4d.jpg -> Prediction: icecream (confidence: 0.0096)
1/1 [==============================] - 0s 375ms/step
pizzavsicecreamdataset/test/pizza/00100_jpg.rf.743db856627866dbc7c6d3af90afd0e2.jpg -> Prediction: icecream (confidence: 0.0210)
Predictions saved to training_results.txt
